In [106]:
import libadalang as lal
from pathlib import Path
from typing import Generator, List, Set

# def get_full_statement_start_line(start: lal.AdaNode) -> int:
#     print(f'{start.kind_name}: {start.sloc_range}')
#     for node in start.parent_chain:
#         if isinstance(node, lal.AdaNodeList) or isinstance(node, lal.CompilationUnit):
#             return node.sloc_range.start.line
#     print(f'Not found: {start}')

def inrange(sloc: lal.Sloc, sloc_range: lal.SlocRange) -> bool:
    start = sloc_range.start
    end = sloc_range.end
    if start.line == end.line == sloc.line:
        return start.column <= sloc.column <= end.column
    elif start.line == sloc.line and start.column > sloc.column:
        return False
    elif end.line == sloc.line and end.column < sloc.column:
        return False
    return start.line <= sloc.line <= end.line

def custom_lookup(node: lal.AdaNode, loc: lal.Sloc) -> lal.AdaNode:
    if len(node.children) == 0: return node
    for child in node.children:
        if child is None: continue
        if inrange(loc, child.sloc_range):
            return custom_lookup(child, loc)
    return node

def is_unpredictable_node(node: lal.AdaNode, loc: lal.Sloc) -> bool:
    if isinstance(node, lal.SubpBody):
        print("Location:", loc)
        print("S:", node.f_subp_spec.sloc_range.end)
        print("E:", node.f_decls.sloc_range.start)
        if loc > node.f_subp_spec.sloc_range.end and loc < node.f_decls.sloc_range.start:
            return True
    for parent in node.parent_chain:
        if isinstance(parent, lal.EndName):
            return True
        if isinstance(parent, lal.CompilationUnit):
            return False
    return False

def get_unpredictable_line_numbers(root: lal.AdaNode, lines: List[str]) -> Set[int]:
    result = set()
    for i, line in enumerate(lines):
        loc = lal.Sloc(i+1, len(line.rstrip('\r\n')))
        node = custom_lookup(root, loc)
        print('Location:', loc)
        print('Lookup:', node)
        print('Parents: ', node.parent_chain)
        if is_unpredictable_node(node, loc):
            result.add(i + 1)
    return result

def create_blocks(file_path: Path) -> Generator[List[str], None, None]:
    with open(str(file_path), "r", encoding="utf-8") as f:
        lines = f.readlines()

    # Remove lines which are just whitespace
    lines = list(filter(lambda line: len(line.strip()) > 0, lines))

    context = lal.AnalysisContext()
    unit = context.get_from_buffer(file_path.name, ''.join(lines))
    root = unit.root
    root.dump()
    unpredictable = get_unpredictable_line_numbers(root, lines)
    print('Unpredictable:', unpredictable)
    for i in range(len(lines)):
        loc = lal.Sloc(i+1, len(lines[i].rstrip('\r\n')))
        res = custom_lookup(root, loc)
        if len(res.children) > 0:
            for child in res.children:
                if child is None: continue
                if test_child := child.lookup(loc) != None:
                    print('Found:', test_child)
        if res is not None:
            print('Found:', res.kind_name, res.sloc_range)
            # start = get_full_statement_start_line(res)
            print('---')
            # print(''.join(lines[start-1:i+1]))
        # sloc = res.sloc_range
        # print(sloc.start, sloc.end)
        # yield [res.text, sloc.start, sloc.end]
    yield None


file_path = Path("/workspaces/ai-indent/data/interim/complete_files/ada/21425_c.ada")



In [107]:
for block in create_blocks(file_path):
    pass

CompilationUnitList 21425_c.ada:1:1-21:7
|item_0:
|  CompilationUnit 21425_c.ada:1:1-3:7
|  |prelude:
|  |  AdaNodeList 21425_c.ada:1:1-1:1
|  |body:
|  |  LibraryItem 21425_c.ada:1:1-3:7
|  |  |has_private:
|  |  |  PrivateAbsent 21425_c.ada:1:1-1:1
|  |  |item:
|  |  |  PackageDecl ["A"] 21425_c.ada:1:1-3:7
|  |  |  |package_name:
|  |  |  |  DefiningName 21425_c.ada:1:9-1:10
|  |  |  |  |name:
|  |  |  |  |  Id "A" 21425_c.ada:1:9-1:10: A
|  |  |  |aspects: None
|  |  |  |public_part:
|  |  |  |  PublicPart 21425_c.ada:1:13-3:1
|  |  |  |  |decls:
|  |  |  |  |  AdaNodeList 21425_c.ada:1:14-2:39
|  |  |  |  |  |item_0:
|  |  |  |  |  |  PragmaNode 21425_c.ada:1:14-1:37
|  |  |  |  |  |  |id:
|  |  |  |  |  |  |  Id "SPARK_Mode" 21425_c.ada:1:21-1:31: SPARK_Mode
|  |  |  |  |  |  |args:
|  |  |  |  |  |  |  BaseAssocList 21425_c.ada:1:33-1:35
|  |  |  |  |  |  |  |item_0:
|  |  |  |  |  |  |  |  PragmaArgumentAssoc 21425_c.ada:1:33-1:35
|  |  |  |  |  |  |  |  |name: None
|  |  |  | 